In [1]:
pwd()

'V:\\UCB Coursework\\Fall23_courses\\Natural Language Processing\\Project\\Emotion-Cause-Analysis'

### Logistic Regression using tf-idf vectorizer

In [2]:

import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load data from JSON
f = open('.\Data\\text\Subtask_1_train.json', encoding="utf-8")
 
# returns JSON object as 
# a dictionary
data = json.load(f)

# Extract texts and emotions
texts = [utterance['text'] for conv in data for utterance in conv['conversation']]
emotions = [utterance['emotion'] for conv in data for utterance in conv['conversation']]

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, emotions, test_size=0.2, random_state=42)

# Initialize and train logistic regression classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Predict on test set
predictions = clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, predictions))


C:\Users\vivek\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vivek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\vivek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


              precision    recall  f1-score   support

       anger       0.39      0.17      0.24       294
     disgust       0.00      0.00      0.00        82
        fear       0.00      0.00      0.00        89
         joy       0.57      0.31      0.41       480
     neutral       0.54      0.89      0.67      1211
     sadness       0.38      0.12      0.18       226
    surprise       0.62      0.46      0.53       342

    accuracy                           0.54      2724
   macro avg       0.36      0.28      0.29      2724
weighted avg       0.49      0.54      0.48      2724



C:\Users\vivek\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vivek\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vivek\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###  Import data and import libraries for usage by the functions

In [2]:
import json
import pandas as pd
from typing import Dict, List
import math
 
# Opening JSON file
f = open('.\Data\\text\Subtask_1_train.json', encoding="utf-8")
 
# returns JSON object as 
# a dictionary
data = json.load(f)

#data

C:\Users\vivek\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vivek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\vivek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Function to make batches

In [3]:
def make_batches(sequences: List[str], batch_size: int) -> List[List[str]]:
    """Yield batch_size chunks from sequences."""

    batch_list=[]

    last_index=len(sequences)-1
    
    for index in range(math.ceil(len(sequences)/batch_size)):
        
        if index+batch_size:
            batch_list.append(sequences[index:index+batch_size])
        else:
            batch_list.append(sequences[index:last_index])
    # DONE
    return batch_list

## Tokenize and padding

In [4]:
from typing import Dict, List, Optional, Tuple
from collections import Counter

import torch
import numpy as np
import spacy



class Tokenizer:
    """Tokenizes and pads a batch of input sentences."""

    def __init__(self, pad_symbol: Optional[str] = "<PAD>"):
        """Initializes the tokenizer

        Args:
            pad_symbol (Optional[str], optional): The symbol for a pad. Defaults to "<PAD>".
        """
        self.pad_symbol = pad_symbol
        self.nlp = spacy.load("en_core_web_sm")
    
    def __call__(self, sentences_batch: List[str]) -> List[List[str]]:
        """Tokenizes each sentence in the batch, and pads them if necessary so
        that we have equal length sentences in the batch.

        Args:
            batch (List[str]): A List of sentence strings

        Returns:
            List[List[str]]: A List of equal-length token Lists.
        """
        sentences_batch = self.tokenize(sentences_batch)
        sentences_batch = self.pad(sentences_batch)

        return sentences_batch

    def tokenize(self, sentences: List[str]) -> List[List[str]]:
        """Tokenizes the List of string sentences into a Lists of tokens using spacy tokenizer.

        Args:
            sentences (List[str]): The input sentence.

        Returns:
            List[str]: The tokenized version of the sentence.
        """
        ret_tokenized_sentences = []
        # for token in self.nlp(sentences):
        #     ret_tokenized_sentences.append(token.text)
            
        ret_tokenized_sentences = []
        for sentence_iter in sentences:
            token_list = ['<SOS>']
            # token_list = []
            for token in self.nlp(sentence_iter):
                token_list.append(token.text)
            token_list.append('<EOS>')
            ret_tokenized_sentences.append(token_list)
        return ret_tokenized_sentences
    
    def pad(self, batch: List[List[str]]) -> List[List[str]]:
        """Appends pad symbols to each tokenized sentence in the batch such that
        every List of tokens is the same length. This means that the max length sentence
        will not be padded.

        Args:
            batch (List[List[str]]): Batch of tokenized sentences.

        Returns:
            List[List[str]]: Batch of padded tokenized sentences. 
        """
        ret_batch = []
        max_len = len(max(batch, key=len))

        ret_batch = [sentence_tokens + ['<P>'] * (max_len - len(sentence_tokens)) for sentence_tokens in batch]

        for sentence in batch:
            for i in range(max_len - len(sentence)):
                sentence.append(self.pad_symbol)

        return batch
        
        # return ret_tokenized_sentences

### Conversion of data to usable format

In [5]:
def data_to_conversation_list(data) -> (List[List[str]], List[List[str]]):
    # Create a list to store the data
    conversation_data = []
    emotion_list = []

    # Iterate through conversations
    for conversation in data:
        conversation_id = conversation["conversation_ID"]
        utterances = conversation["conversation"]
        
        # We make batches now and use those.
        # tokenized_data = []
        # Note: Labels need to be batched in the same way to ensure
        # We have train sentence and label batches lining up.
        #for batch in make_batches(utterances['text'], batch_size):
        #    tokenized_data.append(tokenizer(batch))
        # tokenized_data = my_tokenizer.tokenize(utterances)
        
        # print("After tokenization:", tokenized_data)
        
        emotion_cause_pairs = conversation["emotion-cause_pairs"]

        utterance_data = []
        emotion_temp = []
        # Process each utterance in the conversation
        for utterance in utterances:
            utterance_id = utterance["utterance_ID"]
            #print("Before tokenization:", utterance["text"], "\n After tokenization:", tokenizer(utterance["text"]))
            text = utterance["speaker"] + ": " + utterance["text"]
            #speaker = utterance["speaker"]
            emotion = utterance["emotion"]

            # Append the data to the list
            # conversation_data.append([conversation_id, utterance_id, text, emotion]) #speaker, emotion])
            utterance_data.append(text)
            emotion_temp.append(emotion)
            #print(utterance_data, emotion_temp)
            assert len(utterance_data) == len(emotion_temp)
        conversation_data.append(utterance_data) #speaker, emotion])
        emotion_list.append(emotion_temp)
        assert len(conversation_data) == len(emotion_list)
    
    return conversation_data, emotion_list

## Function to retrieve sentence embeddings

In [6]:
from transformers import BertTokenizer, BertModel
import torch

def get_sentence_embedding(sentence, model_name="bert-base-uncased"):
    # Load pre-trained model tokenizer
    tokenizer = BertTokenizer.from_pretrained(model_name)
    
    # Encode text
    encoded_input = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)

    # Load pre-trained model
    model = BertModel.from_pretrained(model_name)

    # Forward pass, get hidden states
    with torch.no_grad():
        output = model(**encoded_input)

    # Get the embeddings of the [CLS] token (first token), representing the entire sentence
    sentence_embedding = output.last_hidden_state[:, 0, :]

    return sentence_embedding




In [7]:
'''
# Special tokens to be removed
special_tokens = {'<SOS>', '<EOS>', '<PAD>'}

# Joining each sublist into strings and then joining these strings
conversation_string = ' '.join(
    ' '.join(
        ' '.join(word for word in inner_list if word not in special_tokens) 
        for inner_list in sublist
    ) 
    for sublist in conversation_list
)
'''


"\n# Special tokens to be removed\nspecial_tokens = {'<SOS>', '<EOS>', '<PAD>'}\n\n# Joining each sublist into strings and then joining these strings\nconversation_string = ' '.join(\n    ' '.join(\n        ' '.join(word for word in inner_list if word not in special_tokens) \n        for inner_list in sublist\n    ) \n    for sublist in conversation_list\n)\n"

## Logistic Regression using embeddings from Bert model

Do not run as it runs forever 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import torch

# Your existing code for Tokenizer, data processing, and get_sentence_embedding functions...

# Preprocess data and obtain BERT embeddings for each utterance
conversation_data, emotion_list = data_to_conversation_list(data)

# Generate BERT embeddings for each utterance
bert_embeddings = []
for conv in conversation_data:
    conv_embeddings = []
    for utterance in conv:
        # Get BERT embedding for each utterance
        embedding = get_sentence_embedding(utterance)
        conv_embeddings.append(embedding)
    bert_embeddings.append(conv_embeddings)

In [ ]:


# Flatten the list of embeddings for each utterance in a conversation
flat_embeddings = [emb for conv in bert_embeddings for emb in conv]

# Flatten the list of emotions for each utterance in a conversation
flat_emotions = [emo for sublist in emotion_list for emo in sublist]

# Convert embeddings and emotions to numpy arrays
X = torch.cat(flat_embeddings).numpy()  # Convert BERT embeddings to numpy array
y = np.array(flat_emotions)

In [ ]:


# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train logistic regression classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

In [ ]:


# Predict on test set
predictions = clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, predictions))


### Logistic Regression using Word2Vec

In [8]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

# Preprocess data and obtain BERT embeddings for each utterance
conversation_data, emotion_list = data_to_conversation_list(data)

# Train Word2Vec model
#word2vec_model = Word2Vec(sentences=conversation_data, vector_size=100, window=5, min_count=1, workers=4)
# Tokenize sentences into words
tokenized_data = [word_tokenize(" ".join(conv)) for conv in conversation_data]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)


# Generate embeddings for each utterance
word2vec_embeddings = []
for conv in conversation_data:

    conv_embeddings = []
    for utterance in conv:

        tokens = word_tokenize(utterance)


        embedding_intermediate = [word2vec_model.wv[str(word)] for word in tokens if word in word2vec_model.wv]

        embedding = np.mean(pd.Series(embedding_intermediate, dtype=object).fillna(0).tolist(), axis=0)

        conv_embeddings.append(embedding)
    word2vec_embeddings.append(conv_embeddings)

# Flatten the list of embeddings for each utterance in a conversation
flat_embeddings = [emb for conv in word2vec_embeddings for emb in conv]




In [9]:
# Flatten the list of emotions for each utterance in a conversation
flat_emotions = [emo for sublist in emotion_list for emo in sublist]

# Convert embeddings and emotions to numpy arrays
X = np.array(flat_embeddings)
y = np.array(flat_emotions)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# print(X_train, X_test, y_train, y_test)

# Initialize and train logistic regression classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Predict on test set
predictions = clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       anger       0.44      0.23      0.30       294
     disgust       0.00      0.00      0.00        82
        fear       0.75      0.07      0.12        89
         joy       0.46      0.23      0.30       480
     neutral       0.55      0.92      0.69      1211
     sadness       0.32      0.05      0.09       226
    surprise       0.57      0.45      0.50       342

    accuracy                           0.54      2724
   macro avg       0.44      0.28      0.29      2724
weighted avg       0.50      0.54      0.47      2724



C:\Users\vivek\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vivek\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vivek\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Glove embeddings (Uncomment if required)

In [ ]:
# ! wget https://nlp.stanford.edu/data/glove.twitter.27B.zip

In [ ]:
# ! tar -xf glove.twitter.27B.zip

In [ ]:
# ! dir

## Doc2Vec (Use if required)

In [ ]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [ ]:

for utterance_list in conversation_list:
    tagged_data = [TaggedDocument(_d, tags=[str(i)]) for i, _d in enumerate(utterance_list)]

In [ ]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

In [ ]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['1'])

## Encode documents of coversations from pre trained doc2vec model

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

def encode_documents(documents: List[List[List[str]]], model: Doc2Vec) -> torch.FloatTensor:
    """Encode the list of documents using a pre-trained Doc2Vec model.

    Args:
        documents (List[str]): List of documents to encode.
        model (Doc2Vec): Pre-trained Doc2Vec model.

    Returns:
        torch.FloatTensor: Tensor of encoded documents.
    """
    encoded_docs = []
    for conversation in documents:
        conversation_embeddings = [model.infer_vector(tokens, epochs=20) for tokens in conversation]
        encoded_docs.extend(conversation_embeddings)
    return torch.FloatTensor(encoded_docs)



def encode_labels(labels: List[List[str]]) -> List[torch.FloatTensor]:
    """Convert string labels into numeric tensors.

    Args:
        labels (List[List[str]]): Nested list of string labels.

    Returns:
        List[torch.FloatTensor]: List of tensors containing the labels.
    """
    label_dict = {'neutral': 0, 'surprise': 1, 'anger': 2, 'sadness': 3, 'happy': 4}  # Define your label mapping
    encoded_labels = []
    for nested_list in labels:
        # Map string labels to numeric values
        numeric_labels = [label_dict.get(label, -1) for label in nested_list]  # Use -1 as default value
        # Remove -1 values (labels not found in the dictionary)
        numeric_labels = [label for label in numeric_labels if label != -1]
        encoded_labels.append(torch.FloatTensor(numeric_labels))
    return encoded_labels


In [ ]:
# Encode conversations
encoded_conversations = encode_documents(conversation_list, model)

# Emotion labels encoding remains the same as before
encoded_emotion_labels = encode_labels(emotion_list)

## Get OOVs from the model

In [ ]:
# Get the vocabulary list
vocab_list = list(model.wv.index_to_key)

In [ ]:
from typing import List

def get_oovs(vocab_list: List[str], model:Doc2Vec) -> List[str]:
    """Find the words in vocab_list that are out-of-vocabulary (OOV) in the Doc2Vec model.
    
    Args:
    vocab_list (List[str]): List of words in your vocabulary.
    model: Pre-trained Doc2Vec model.
    
    Returns:
    List[str]: List of out-of-vocabulary words in vocab_list.
    """
    oov_words = [word for word in vocab_list if word not in model.wv]
    return oov_words
